# Étape 1 : Prérequis

In [7]:
# Vérification du GPU et installation des dépendances nécessaires
!nvidia-smi
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers


Wed Aug 21 22:46:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.80                 Driver Version: 546.80       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   40C    P8               4W /  70W |    203MiB /  8188MiB |     24%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Could not find platform independent libraries <prefix>
Could not find platform independent libraries <prefix>
Could not find platform independent libraries <prefix>
Could not find platform independent libraries <prefix>
Could not find platform independent libraries <prefix>
Could not find platform independent libraries <prefix>


In [8]:
# Afficher les versions des bibliothèques installées
!nvidia-smi
!echo "Version de bitsandbytes :"
!pip show bitsandbytes
!echo "Version de peft :"
!pip show peft
!echo "Version de trl :"
!pip show trl
!echo "Version de accelerate :"
!pip show accelerate
!echo "Version de datasets :"
!pip show datasets
!echo "Version de transformers :"
!pip show transformers

Wed Aug 21 22:46:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.80                 Driver Version: 546.80       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   42C    P8               4W /  70W |    203MiB /  8188MiB |     33%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Could not find platform independent libraries <prefix>


"Version de peft :"
Name: peft
Version: 0.12.0
Summary: Parameter-Efficient Fine-Tuning (PEFT)
Home-page: https://github.com/huggingface/peft
Author: The HuggingFace team
Author-email: sourab@huggingface.co
License: Apache
Location: c:\test\.venv\Lib\site-packages
Requires: accelerate, huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch, tqdm, transformers
Required-by: 


Could not find platform independent libraries <prefix>


"Version de trl :"
Name: trl
Version: 0.9.6
Summary: Train transformer language models with reinforcement learning.
Home-page: https://github.com/huggingface/trl
Author: Leandro von Werra
Author-email: leandro.vonwerra@gmail.com
License: Apache 2.0
Location: c:\test\.venv\Lib\site-packages
Requires: accelerate, datasets, numpy, torch, transformers, tyro
Required-by: 


...

Could not find platform independent libraries <prefix>


"Version de datasets :"
Name: datasets
Version: 2.21.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: c:\test\.venv\Lib\site-packages
Requires: aiohttp, dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, tqdm, xxhash
Required-by: trl


Could not find platform independent libraries <prefix>


"Version de transformers :"
Name: transformers
Version: 4.44.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: c:\test\.venv\Lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, trl


Could not find platform independent libraries <prefix>


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Configuration pour la quantification en 4 bits
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [10]:
# Si vous utilisez Google Colab, connectez-vous à Hugging Face
from huggingface_hub import notebook_login
notebook_login()

# Étape 2 : Chargement du modèle
Chargement du modèle avec la configuration de quantification QLoRA pour réduire l'utilisation de la mémoire

In [13]:
torch.cuda.is_available()

False

In [14]:
model_id = "google/gemma-2-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"": 0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

model-00001-of-00002.safetensors:  12%|#1        | 577M/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Some parameters are on the meta device device because they were offloaded to the disk and cpu.


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [18]:
# Fonction pour générer une réponse à partir du modèle
def obtenir_reponse(query: str, model, tokenizer) -> str:
    device = "cpu"

    template_prompt = """
    <start_of_turn>utilisateur
    Ci-dessous se trouve une instruction qui décrit une tâche. Écrivez une réponse qui complète correctement la demande.
    {query}
    <end_of_turn>\n<start_of_turn>modèle
    """
    prompt = template_prompt.format(query=query)

    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encodeds.to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return decoded

In [19]:
# Exemple de test du modèle
resultat = obtenir_reponse(query="écrire des tests unitaires pour cette fonction def add(a,b): return a+b en utilisant pytest", model=model, tokenizer=tokenizer)
print(resultat)


    utilisateur
    Ci-dessous se trouve une instruction qui décrit une tâche. Écrivez une réponse qui complète correctement la demande.
    écrire des tests unitaires pour cette fonction def add(a,b): return a+b en utilisant pytest
    
modèle
    ```python
from typing import Tuple

def add(a: int, b: int) -> int:  
    return a + b

import pytest

@pytest.mark.parametrize("a, b, expected", [
    (1, 2, 3),
    (-5, 3, -2),
    (0, 0, 0),
])
def test_add(a, b, expected):
   assert add(a, b) == expected 

```

**Explication du code:**

1. **Importation des bibliothèques nécessaires:**  `pytest` est le framework de testing unitaires que nous allons utiliser. L'import `typing` permet de documenter le type des paramètres.
2. **Définition de la fonction `add`:** Cette fonction prend deux arguments et retourne leur somme. 
3. **Définition de tests unitaires à l'aide de `@pytest.mark.parametrize`:**
   * `@pytest.mark.parametrize`: Cette annotation est utilisée pour indiquer que plusieurs c

# Étape 3 : Chargement du dataset pour le fine-tuning



Nous utiliserons ce [dataset](https://huggingface.co/datasets/TokenBender/code_instructions_122k_alpaca_style) qui est un excellent point de départ pour entraîner des modèles de génération de code.

In [20]:
# from datasets import load_dataset

# dataset = load_dataset("Noorgha/pytest", split="train")
# dataset

In [23]:
from datasets import load_dataset

# Charger les données à partir du fichier data.json
dataset = load_dataset("json", data_files="formatted_minidata_with_integration_instructions.json", split="train")
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 981
})


In [24]:
# Transformation des données en prompts adaptés à l'entraînement
def generate_prompt(data_point):
    prefix_text = 'Below is an instruction that describes a task. Write a pytest unit test function that ' \
                  'appropriately completes the request.\n\n'
    if data_point['input']:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} Here is the function:\n{data_point["input"]} <end_of_turn>\n<start_of_turn>model\n{data_point["output"]} <end_of_turn>"""
    else:
        text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} <end_of_turn>\n<start_of_turn>model\n{data_point["output"]} <end_of_turn>"""
    return text

In [25]:
# Ajouter la colonne "prompt" au dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

In [26]:
# Mélanger et diviser le dataset en ensembles d'entraînement et de test
dataset = dataset.shuffle(seed=1234)
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]
print(train_data)
print(test_data)

Map:   0%|          | 0/981 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 784
})
Dataset({
    features: ['instruction', 'input', 'output', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 197
})


# Étape 4 : Application de LoRA (Low-Rank Adaptation)
Application de la technique LoRA pour optimiser l'entraînement du modèle


In [27]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

# Activation du gradient checkpointing pour optimiser la mémoire
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [34]:
import bitsandbytes as bnb

# Identification des modules linéaires du modèle pour LoRA
def trouver_tous_noms_lineaires(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
        if 'lm_head' in lora_module_names:
            lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [33]:
modules = trouver_tous_noms_lineaires(model)
print(modules)

NameError: name 'lora_module_names' is not defined

In [ ]:
# Configuration de LoRA
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
# Application de la configuration LoRA au modèle
model = get_peft_model(model, lora_config)
trainable, total = model.get_nb_trainable_parameters()
print(f"Paramètres entraînables : {trainable} | total : {total} | Pourcentage : {trainable/total*100:.4f}%")


ValueError: Target modules set() not found in the base model. Please check the target modules and try again.

# Étape 5 : Entraînement du modèle
Fine-tuning du modèle avec qLora et Supervised Fine-Tuning (SFT)

In [ ]:
import matplotlib.pyplot as plt
from transformers import TrainerCallback
from datasets import load_metric
import trl
from trl import SFTTrainer
import transformers

# Callback pour capturer l'accuracy après chaque epoch
class AccuracyCallback(TrainerCallback):
    def __init__(self):
        self.train_accuracies = []
        self.eval_accuracies = []
        self.metric = load_metric("accuracy")

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        # Capturer l'accuracy après chaque epoch
        if metrics is not None:
            eval_accuracy = metrics.get("eval_accuracy")
            if eval_accuracy is not None:
                self.eval_accuracies.append(eval_accuracy)

    def on_log(self, args, state, control, logs=None, **kwargs):
        # Capturer l'accuracy sur les données d'entraînement
        if logs is not None:
            train_loss = logs.get("loss")
            if train_loss is not None and state.is_local_process_zero:
                self.train_accuracies.append(train_loss)

# Initialiser le callback
accuracy_callback = AccuracyCallback()

In [ ]:
# Configuration de l'entraîneur avec le callback pour capturer les accuracies
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        evaluation_strategy="epoch",  # Effectuer une évaluation après chaque epoch
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[accuracy_callback],
)

In [ ]:
# Lancement de l'entraînement avec évaluation après chaque epoch
trainer.train()

In [ ]:
# Étape 2 : Tracer les courbes d'accuracy
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(accuracy_callback.train_accuracies) + 1), accuracy_callback.train_accuracies, label='Accuracy Entraînement')
plt.plot(range(1, len(accuracy_callback.eval_accuracies) + 1), accuracy_callback.eval_accuracies, label='Accuracy Évaluation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Courbes d\'Accuracy au cours de l\'Entraînement')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Partage des adaptateurs sur le Hub Hugging Face
nouveau_modele = "model-aftaitesting-2b" # Nom du modèle que vous allez pousser sur le Hub Hugging Face
trainer.model.save_pretrained(nouveau_modele)


In [ ]:
# Fusion et sauvegarde du modèle
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

merged_model = PeftModel.from_pretrained(base_model, nouveau_modele)
merged_model = merged_model.merge_and_unload()

In [ ]:
# Sauvegarder le modèle fusionné
merged_model.save_pretrained("merged_model", safe_serialization=True)
tokenizer.save_pretrained("merged_model")

In [ ]:
# Pousser le modèle et le tokenizer sur le Hub Hugging Face
merged_model.push_to_hub(nouveau_modele, use_temp_dir=False)
tokenizer.push_to_hub(nouveau_modele, use_temp_dir=False)

In [ ]:
# Tester le modèle fine-tuné
resultat = obtenir_reponse(query="écrire simplement du code de tests unitaires en utilisant pytest pour cette fonction : def add(a,b): return a+b avec pytest. Donner juste le code, sans instruction", model=merged_model, tokenizer=tokenizer)
print(resultat)